# demonstration of distributed ranking/sorting with Elo

simulation of a distributed ranking system for a large number of items using a number of unreliable users, and rewarding users with reputation

*The Social Network* references the 'chess ranking algorithm' Elo rating as the sorting mechanism of Facemash

this was motivated by the following post:

https://stackoverflow.com/questions/164831/how-to-rank-a-million-images-with-a-crowdsourced-sort

### The Elo System

https://www.geeksforgeeks.org/elo-rating-algorithm/

1. new players start with rating of 1600
2. win_probability = 1/(1 + 10^((opponent_rating – player_rating)/400))
3. score_point = 1 point if they win the match, 0 if they lose, and 0.5 for a draw.
4. player_new_rating = player_rating + (K_value * (score_point – win_probability))

### Features

- distributed: nothing depends on caching information; every choice is made at the time of choosing
- asynchronous: while not demonstrated here, there is no time-dependent order to ranking
- dual-ranking: maintains sample score and reputation score together

### Rejected Alternatives

Bubble or Merge Sort: relies on users to respond in timely fashion, as these are sequenced  
Majority-Wins Reputation (like Papago Gym): relies on caching results over time period  
Swiss Tournament Pairing: relies on caching pairs; can't add new samples until one session is over  

### Parameters

`getpairing() n` : `n` controls the number of lowest-seen items to select from  
`Crowdscorer noise` : the *un*reliability of each user, as a list (where unreliablility = prob of choosing wrong)  
`Crowdscorer reward` : how much to adjust reputations up/down based on expected score (linear)  
`Crowdscorer temperature` : adjust how flat the probability sampling is over the users (higher = flatter)  
`updateELO() k` : how 'swingy' is the ELO adjustment. higher = bigger adjustments. for chess, k=32 or k=20 is normal  
`crowdscoring() users` : controls how many users are grading (related to `Crowdscoreer noise`)

### Todo

adjust reputation adjustment; right now the score is never normalized so it just keeps going higher. marginal reputation update could be inversely proportional to total number of samples judged, or something... 

In [1]:
import numpy as np
import heapq

## task: sorting integers

for this toy example, we will represent each sample as an integer, with the goal being to choose the higher integer between two.

In [2]:
# initialize a random sequence of integers
num_samples = 10000
max_value   = 50000

# initialize data to random values
data = np.random.randint(1, max_value, num_samples)
# initialize elo scores to 1600 +/- 100
elos = [1600 + np.random.randint(-100, 100) for _ in range(num_samples)]
# initialize # of validations to 0
hits = [0 for _ in range(num_samples)]
# check lens
len(data), len(elos), len(hits)

(10000, 10000, 10000)

In [3]:
# fn to rank two lists by one list
def rank_x_by_y(a, b):
    results = list(zip(a, b))
    results = sorted(results, key=lambda x: x[1], reverse=True)
    srt_a = [t[0] for t in results]
    srt_b = [t[1] for t in results]
    return srt_a, srt_b

### initial rankings are random

here we check the initial distribution by ranking according to the randomized initial Elo scores.

we can see that the distribution is random

In [4]:
# top scores
init_data, init_elos = rank_x_by_y(data, elos)
print("top ranked:")
print("elo", '\t', "value")
for i in range(10):
    print(init_elos[i], '\t', '{: 5.0f}'.format(init_data[i]))
print("\nlowest ranked:")
print("elo", '\t', "value")
for i in reversed(range(10)):
    print(init_elos[-i-1], '\t', '{: 5.0f}'.format(init_data[-i-1]))

top ranked:
elo 	 value
1699 	  20001
1699 	  25079
1699 	  49798
1699 	  27254
1699 	  40051
1699 	  20652
1699 	  7189
1699 	  44127
1699 	  27900
1699 	  8214

lowest ranked:
elo 	 value
1500 	  49175
1500 	  30415
1500 	  48409
1500 	  27463
1500 	  7299
1500 	  6507
1500 	  15158
1500 	  23928
1500 	  8337
1500 	  49828


### iterative scoring

while scoring:
1. choose (*similarly-ranked*) sents from the less-seen ones
2. display to "user" who chooses best, with noise (sometimes wrong)
3. update Elo and hit count

In [5]:
# choose pair helper function
# this returns INDICES (not values) of pair
# method: choose randomly from the n least-seen items, then get closest scrore
def getpairing(hitcounts, eloscores, n=100):
    # get n indices of lowest-seen
    low = np.argpartition(hitcounts, n)[:n]
    # sample random index
    idx = np.random.choice(low, 1, replace=False)[0]
    # get score differences
    elodf = np.ma.array(np.abs(np.array(eloscores)[low]-np.array(eloscores)[idx]), mask=False)
    elodf.mask[low.tolist().index(idx)] = True
    bdx = low[np.argmin(elodf)]
    # return
    # return idx[0], idx[1]
    return idx, bdx

In [6]:
# test
for i in range(5):
    a, b = getpairing(hits, elos)
    print(a, elos[a], b, elos[b])

6679 1648 6688 1648
6646 1637 6701 1638
6582 1501 6662 1501
6666 1584 6710 1582
6704 1572 6663 1573


## simulate a crowd of unreliable scorers

initialize a number of users, specified by their reliability (noise = % that they guess wrong)

when scoring, assign to a user based on their reputation

reward/penalize user reputations based on *expected* result == better current Elo score

reward policy is linear (don't scale up/down according to # of times that the users have been chosen)

temperature is softmax temperature for prob dist, bigger = flatter

hypothesis: this correlated with true reliability (1.0 - noise) ??

In [7]:
# choose best with noise function
# this should be VALUES (not indices of data)
# noise is the % of unreliability
def choosebest(v1, v2, noise=0.25):
    choices = [v1, v2]
    wrong = (noise > np.random.random())
    if not wrong:
        return choices.index(max(choices))
    else:
        return choices.index(min(choices))

In [22]:
# test the default 25% - should be wrong about 2~3 times on average
for i in range(10):
    v1, v2 = getpairing(hits, elos) # < this is 'wrong' for real task, just testing
    idx = choosebest(v1, v2)
    choices = [v1, v2]
    if choices.index(max(choices)) == idx:
        tru = "True"
    else:
        tru = "False"
    print(v1, v2, idx, tru)

6583 6641 1 True
6659 6682 1 True
6584 6694 0 False
6569 6676 1 True
6591 6704 1 True
6715 6693 0 True
6705 6693 0 True
6656 6597 1 False
6585 6600 1 True
6587 6665 1 True


In [9]:
# group rescorer class
# this simulates a number of graders
# each grader has their own "reputation" using the same ELO system
class Crowdscorer:
    
    def __init__(self, noises, reward=0.01, temperature=10):
        self.noises=noises
        self.reward=reward
        self.temp=temperature
        self.reputations=[1.0 for _ in range(len(noises))]
        self.selected=[0 for _ in range(len(noises))]
        
    def _choosebest(self, v1, v2, noise=0.0):
        choices = [v1, v2]
        wrong = (noise > np.random.random())
        if not wrong:
            return choices.index(max(choices))
        else:
            return choices.index(min(choices))
        
    def score(self, v1, v2):
        # get 'true' based on prior
        prior = self._choosebest(v1, v2, noise=0.0)
        # get sampling proabability based on reputation
        e_x = np.exp(np.array(self.reputations)/self.temp)
        prb = e_x/e_x.sum()
        # select user
        idx = np.random.choice([i for i in range(len(self.noises))], p=prb)
        # get user choice
        choice = self._choosebest(v1, v2, noise=self.noises[idx])
        # reward if agree with prior, penalize if against prior
        if prior == choice:
            self.reputations[idx] += self.reward
            self.selected[idx] += 1
        else:
            self.reputations[idx] -= self.reward
            self.selected[idx] += 1
        return choice
    
    def getreputations(self):
        return self.noises, self.reputations, self.selected

In [10]:
test = Crowdscorer([0.25, 0.5, 0.66])

In [11]:
# test the class version, with multiple users
for i in range(10):
    v1, v2 = getpairing(hits, elos) # < this is 'wrong' for real task, just testing
    idx = test.score(v1, v2)
    choices = [v1, v2]
    if choices.index(max(choices)) == idx:
        tru = "True"
    else:
        tru = "False"
    print(v1, v2, idx, tru)

6675 6709 1 True
6674 6712 0 False
6664 6645 0 True
6595 6676 1 True
6648 6713 1 True
6595 6676 1 True
6644 6592 1 False
6699 6700 0 False
6596 6717 1 True
6705 6693 1 False


In [12]:
# update Elo function
# this should be Elo scores, and index of winner
# k is a parameter that can be graded (higher for newbies)
# we set it relatively high, bc only ranking least-seen elements
# PS yes it is 'Elo' not 'ELO' but i like electric light orchestra, what can i say
def updateELO(elo1, elo2, win, k=32):
    # win probabilities
    e1prob = 1.0/(10**((elo2-elo1)/400.)+1)
    e2prob = 1.0/(10**((elo1-elo2)/400.)+1)
    # score update - flip it for first pos
    e1scor = abs(1-win)
    e2scor = win
    # rating update
    e1new = elo1 + (k*(e1scor-e1prob))
    e2new = elo2 + (k*(e2scor-e2prob))
    return e1new, e2new

In [13]:
# test some cases
print(updateELO(1600, 1600, 0)) # even newbie, A wins
print(updateELO(1600, 1600, 1)) # even newbie, B wins
print(updateELO(2400, 1600, 0)) # better player wins
print(updateELO(2400, 1600, 1)) # upset: newbie wins
print(updateELO(1600, 2400, 0)) # rev: upset: newbie wins
print(updateELO(1600, 2400, 1)) # rev: better player wins

(1616.0, 1584.0)
(1584.0, 1616.0)
(2400.3168316831684, 1599.6831683168316)
(2368.3168316831684, 1631.6831683168316)
(1631.6831683168316, 2368.3168316831684)
(1599.6831683168316, 2400.3168316831684)


## iterative scoring

while number of trials not reached:
1. get a pairing according to the pairing policy
2. have somewhat-unreliable scorer score the pair
3. update the ELO scores and increment hit count

In [14]:
def crowdscoring(data, elos, hits, users=10, n=10000, k=32):
    # copy new instances of data
    t_data = data[:]
    t_elos = elos[:]
    t_hits = hits[:]
    # initialize scorers
    noises  = [np.random.random()/1.5 for user in range(users)]
    scorer = Crowdscorer(noises, reward=0.005, temperature=6)
    # for each trial...
    c = 1
    for trial in range(n):
        # get INDEX pairing
        idx1, idx2 = getpairing(t_hits, t_elos)
        # pass VALUES to "user" to score
        win_idx = scorer.score(t_data[idx1], t_data[idx2])
        # update hit count upon score receipt
        t_hits[idx1] += 1
        t_hits[idx2] += 1
        # update ELOs
        new1, new2 = updateELO(t_elos[idx1], t_elos[idx2], win_idx, k=k)
        t_elos[idx1] = new1
        t_elos[idx2] = new2
        if trial > 0 and (trial+1) % int(n/10) == 0:
            print((trial+1), "trials done:", c*10, '%...')
            c += 1
    return t_data, t_elos, t_hits, scorer

In [15]:
end_data, end_elos, end_hits, scorer = crowdscoring(data, elos, hits, users=50, n=100000, k=50)

10000 trials done: 10 %...
20000 trials done: 20 %...
30000 trials done: 30 %...
40000 trials done: 40 %...
50000 trials done: 50 %...
60000 trials done: 60 %...
70000 trials done: 70 %...
80000 trials done: 80 %...
90000 trials done: 90 %...
100000 trials done: 100 %...


### "final" scores will be ranked according to ELO

...though degree of success relies on number of trial, user reliability, *k* parameter

In [16]:
# top scores
srt_data, srt_elos = rank_x_by_y(end_data, end_elos)
print("top ranked:")
print("elo", '\t', "value")
for i in range(10):
    print('{:04.0f}'.format(srt_elos[i]), '\t', '{: 5.0f}'.format(srt_data[i]))
print("\nlowest ranked:")
print("elo", '\t', "value")
for i in reversed(range(10)):
    print('{:04.0f}'.format(srt_elos[-i-1]), '\t', '{: 5.0f}'.format(srt_data[-i-1]))

top ranked:
elo 	 value
2061 	  49714
2033 	  49687
2023 	  48429
2017 	  49726
2015 	  49858
2011 	  49385
2006 	  49724
1994 	  49623
1991 	  49527
1983 	  49245

lowest ranked:
elo 	 value
1214 	   372
1210 	   311
1204 	   141
1189 	   444
1185 	    48
1184 	   228
1181 	    79
1173 	   457
1160 	   328
1139 	   744


## examine whether expected choice is reliable reputation factor

using expected outcome could be a problem with many new data, as new data is initialized at a 'neutral' score, so the 'expected' selections may be counter to the true answers?

but in this toy demo, it appears that this is an acceptable scoring method, as reputation and times chosen correlate to reliability, and (with proper temperature adjustment), the selection probability does not _entirely_ shut out trolling users, while still favoring reliable scorers. this does have the chance to converge into a vicious cycle rewarding early adopters by favoring those with higher reputation. so this also needs to be taken into account.

of course, in a real system, the 'reliability' is subjective and is a latent variable; it is only because of this task's artificiality that we can objectively peek at the "true" underlying reliability. and due to the fact that reputation will fluctuate, some smoothing factor should be implemented so that users are not constantly wondering why their reputation is going up and down.

also, notice that this took ten times as many validations as samples; reducing `n` demonstrates that fewer iterations do not dependably sort the items.

In [17]:
n, r, s = scorer.getreputations()
n = [1-x for x in n]

In [18]:
print('reliability\t\treputation\t\ttimes chosen')
for t in sorted(list(zip(n, r, s)), key=lambda x:x[0], reverse=True):
    print(t[0], '\t', t[1], '\t', t[2])

reliability		reputation		times chosen
0.9973648829006753 	 32.58499999999822 	 6347
0.9967286113438104 	 28.044999999998705 	 5445
0.9960696739674756 	 71.82000000000717 	 14274
0.9745095134622639 	 18.380000000000628 	 3624
0.9380588610845417 	 14.0700000000008 	 2948
0.9260490602275694 	 11.155000000000344 	 2419
0.9195511440530173 	 11.885000000000458 	 2553
0.9120009073592603 	 10.665000000000267 	 2319
0.9050991212695274 	 10.195000000000194 	 2317
0.8964566116872507 	 10.43000000000023 	 2366
0.8736832262721358 	 8.934999999999997 	 2157
0.8552839029406051 	 8.239999999999888 	 2090
0.8315932055004788 	 8.414999999999916 	 2147
0.8286462955629535 	 7.744999999999856 	 2009
0.8121199091966981 	 6.314999999999887 	 1819
0.808346785761692 	 6.494999999999883 	 1801
0.7627688571199326 	 5.2449999999999095 	 1585
0.7470668273700631 	 4.954999999999916 	 1549
0.7282694709446265 	 4.579999999999924 	 1530
0.6986021648238361 	 3.9799999999999365 	 1578
0.6948254552374578 	 3.684999999999